In [1]:
from rdflib import Graph,Literal,RDF,URIRef, RDFS, DC, DCTERMS, DCAT
from rdflib.namespace import XSD, FOAF
from rdflib import Namespace
import pandas as pd
from sodapy import Socrata

In [53]:
g = Graph()
g1 = Graph()

In [54]:
g.parse('file:///Users/rishimidha/Desktop/Ontology/NSdiseaseontology.owl')

<Graph identifier=Nb31d4842116a40999f46694ff7216e36 (<class 'rdflib.graph.Graph'>)>

In [55]:
g1.parse('file:///Users/rishimidha/Desktop/Ontology/doid.owl')

<Graph identifier=Na503082b60384d61bbe6b093d8345b88 (<class 'rdflib.graph.Graph'>)>

In [69]:
cube = Namespace("http://purl.org/linked-data/cube#")
sdmxdimension = Namespace("http://purl.org/linked-data/sdmx/2009/dimension#")
geo = Namespace ("https://www.geonames.org/")
sdmxcode = Namespace ("http://purl.org/linked-data/sdmx#")
freq = Namespace ("http://purl.org/cld/freq/")
doid = Namespace ("http://purl.obolibrary.org/obo/DOID#")
doid_4 = URIRef("http://purl.obolibrary.org/obo/DOID_4")

In [57]:
for s, p, o in g.triples((None,sdmxdimension.refArea,None)):
    print(s,p,o)




http://www.semanticweb.org/rmidha/ontologies/statistical#obs-HIVAlberta-6 http://purl.org/linked-data/sdmx/2009/dimension#refArea http://www.w3.org/2003/01/geo/wgs84_pos#5883102
http://www.semanticweb.org/rmidha/ontologies/statistical#obs-HIVAlberta-3 http://purl.org/linked-data/sdmx/2009/dimension#refArea http://www.w3.org/2003/01/geo/wgs84_pos#5883102
http://www.semanticweb.org/rmidha/ontologies/statistical#obs-HIVAlberta-4 http://purl.org/linked-data/sdmx/2009/dimension#refArea http://www.w3.org/2003/01/geo/wgs84_pos#5883102
http://www.semanticweb.org/rmidha/ontologies/statistical#obs-HIVAlberta-7 http://purl.org/linked-data/sdmx/2009/dimension#refArea http://www.w3.org/2003/01/geo/wgs84_pos#5883102
http://www.semanticweb.org/rmidha/ontologies/statistical#obs-HIVAlberta-8 http://purl.org/linked-data/sdmx/2009/dimension#refArea http://www.w3.org/2003/01/geo/wgs84_pos#5883102
http://www.semanticweb.org/rmidha/ontologies/statistical#obs-HIVAlberta-5 http://purl.org/linked-data/sdmx/200

In [58]:
list_of_api={"sr7h-uyxt",    #Tetanus
             "naay-xy8i",    #AIDS
             "5kjg-qidk",    #Salmonellosis
             "r9i7-qnuh",    #HepatitisA
             "w7bm-jav4",    #HepatitisC
             "s4zv-fef7",    #Shigellosis
             "dxhy-v63n",    #Giardiasis
             "sps3-eq7e",    #Chlamydia
             "fbx9-xzkq",    #Legionellosis
             "enig-mjwt",    #Cyclosporiasis
             "5itt-58m2",    #Pertussis
             "4p4c-dv5x",    #Malaria
             "tsr3-8hh2",    #HIV
             "qsy8-u73r",    #Gonorrhea
             "nn2f-jgv4",    #Campylobacteriosis
             "4vij-qbup",    #Botulism
             "5mqu-g4g2",    #Rubella
             "48h6-dubm",    #Tuberculosis
             "e2eh-63yz",    #Mumps
             "p42z-mdp9",    #Typhoid
             "9ckr-t936",    #Cryptosporidiosis
            }

In [138]:
client = Socrata("data.novascotia.ca", None)
for i in list_of_api:
    results = client.get(i)
    results1 = client.get_metadata(i)
    results_df = pd.DataFrame.from_records(results)
    stat = Namespace('http://www.semanticweb.org/rmidha/ontologies/statistical#')
    name=results1["name"].replace(' ','')
    name=Literal(name, datatype="str")
    comment=Literal(results1["description"],datatype="str")
    license=URIRef("https://novascotia.ca/opendata/licence.asp")
    #created=Literal()
    Category=results1["category"]
    Category=Category.replace(" ","")
    Category=stat+Category
    theme=URIRef(Category)
    urlhealthandwellness="https://data.novascotia.ca/Health-and-Wellness/"
    urldataset=urlhealthandwellness+name+'/'+results1["id"]
    isDefinedBy=Literal(urldataset)
    Dept=results1["metadata"]["custom_fields"]["Detailed Metadata"]["Department"]
    Dept=Dept.replace(" ","")
    Dept=stat+Dept
    department=URIRef(Dept)
    DSD=URIRef("http://www.semanticweb.org/rmidha/ontologies/statistical#dsd-diseasedatasets")
    lang=results1["metadata"]["custom_fields"]["Detailed Metadata"]["Language"]
    if len(lang)== 3:
        lang="http://www.lexvo.org/page/iso639-3/"+lang
    lang=URIRef(lang)
    frequency=results1["metadata"]["custom_fields"]["Detailed Metadata"]["Frequency"]
    if frequency=="Annual" or frequency=="Annually"or frequency=="annual"or frequency=="annually"or frequency=="yearly"or frequency=="Yearly":
        frequency=freq.annual
    frequency=URIRef(frequency)
    Location=results1["metadata"]["custom_fields"]["Detailed Metadata"]["Geographic Region Name"]
    spatial=geo+"6091530"
    spatial=URIRef(spatial)
    dataset=stat+'dataset-'+name
    dataset= URIRef(dataset)
    g.add((dataset,RDF.type,cube.DataSet))
    g.add((dataset,RDFS.label,name))
    g.add((dataset,RDFS.comment,comment))
    g.add((dataset,DCTERMS.license,license))
    g.add((dataset,DCAT.theme,theme))
    for i in results1["tags"]:
        tag=i
        tag=tag.replace(" ","")
        new=stat+tag
        keyword=URIRef(new)
        g.add((dataset,DCAT.keyword,keyword))
    g.add((dataset,RDFS.isDefinedBy,isDefinedBy))
    g.add((dataset,stat.department,department))
    g.add((dataset,cube.structure,DSD))
    g.add((dataset,DCTERMS.language,lang))
    g.add((dataset,DCTERMS.accrualPeriodicity,frequency))
    g.add((dataset,DCTERMS.spatial,spatial))
    for i in range(1,len(results_df)):
        x=str(i)
        obsiri=stat+"obs-"+name+"-"+x
        obs=URIRef(obsiri)
        g.add((obs,cube.dataSet,dataset))
        g.add((obs,RDF.type,cube.Observation))
        yr=str(results_df["year"][i])
        yr='https://dbpedia.org/page/'+yr
        year=URIRef(yr)
        g.add((obs,sdmxdimension.refPeriod,year))
        regioniri=geo+"6091530"
        region=URIRef(regioniri)
        g.add((obs,sdmxdimension.refArea,region))
        T=sdmxcode.sex+"-T"
        T = URIRef(T)
        g.add((obs,sdmxcode.sex,T))
        numberofcases=Literal(results_df["number_of_cases"][i],datatype="int")
        g.add((obs,stat.numberofcases,numberofcases))
        rate_per_100_000_population=Literal(results_df["rate_per_100_000_population"][i],datatype="float")
        g.add((obs,stat.rateper100kpopulation,rate_per_100_000_population))
        disease=results_df["disease"][i]
        disease=disease.lower()
        if (disease=="human immunodeficiency virus"):
            disease="human immunodeficiency virus infectious disease"
        elif (disease=="typhoid"):
            disease="typhoid fever"
        elif (disease=="acquired immune deficiency syndrome"):
            disease="acquired immunodeficiency syndrome"
        elif (disease=="hepatitis c"):
            disease="hepatitis C"
        elif (disease=="hepatitis a"):
            disease="hepatitis A"
        diseaselabel=Literal(disease)
        for (s, p, o) in g1.triples((None,RDFS.label,diseaselabel)):
            diseaseiri=URIRef(s)
            g.add((obs,stat.hasdisease,diseaseiri))
            while(diseaseiri!=doid_4):
                x=g1.value(diseaseiri,RDFS.subClassOf)
                diseaseiri=URIRef(x)
                g.add((obs,stat.hasdisease,diseaseiri))
       
                
                

In [141]:
  
file = open("output.txt", "w")   
g.serialize('output_file_final.ttl',format='ttl')

